In [ ]:
# Step 1: Mount Google Drive (optional, if the .rar file is stored on Drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms
import os

In [ ]:
!cp /content/drive/MyDrive/posture_screen/Database/labels.csv /content/Total_datalabels_onehot.csv

In [ ]:
!cp -r /content/drive/MyDrive/posture_screen/Database/images /content/images

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms
import os

# Check for device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1: Define the Custom Dataset Class
class ImageLabelDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None, max_fraction=0.5):
        self.data = pd.read_csv(csv_file)  # Assuming CSV has headers "image_name" and "labels"
        self.image_dir = image_dir
        self.transform = transform
        self.balance_dataset(max_fraction)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx]['IMG_NAME'])  # Image name
        image = Image.open(img_name).convert("RGB")
        print(img_name)

        # Convert label string to list of integers
        labels = self.data.iloc[idx]['Label']
        labels = [int(x) for x in labels[1:-1].split(',')]  # Split space-separated integers
        labels = np.array(labels, dtype=np.float32)  # Convert to NumPy array
        print(labels)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(labels)

    def balance_dataset(self, max_fraction=0.5):
        # Compute class frequencies
        labels_all = self.data['Label'].apply(lambda x: np.array([int(i) for i in x[1:-1].split(',')]))
        label_sums = np.sum(np.stack(labels_all), axis=0)
        max_samples = int(max_fraction * max(label_sums))

        balanced_data = []
        for _, row in self.data.iterrows():
            labels = np.array([int(i) for i in row['Label'][1:-1].split(',')])
            repetitions = min(int(max_samples / (sum(labels) + 1e-5)), 5)
            balanced_data.extend([row] * repetitions)

        self.data = pd.DataFrame(balanced_data)

# Step 2: Define Image Transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.485, 0.485], std=[0.229, 0.229, 0.229]),
])

# Step 3: Define the Model with MobileNetV2 and Dropout Layer
class ModifiedMobileNet(nn.Module):
    def __init__(self):
        super(ModifiedMobileNet, self).__init__()
        self.model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)

        # Replace the classifier with a new one for multi-label classification
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 12)
        )

    def forward(self, x):
        return self.model(x)

# Step 4: Define Focal Loss with Class Weights
class FocalLoss(nn.Module):
    def __init__(self, gamma=2., alpha=0.25, class_weights=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.class_weights = class_weights

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(weight=self.class_weights)(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss.mean()

# Step 5: Load Data and Create DataLoader
csv_file = '/content/Total_datalabels_onehot.csv'
image_dir = '/content/images'

# Initialize dataset
dataset = ImageLabelDataset(csv_file=csv_file, image_dir=image_dir, transform=transform, max_fraction=0.5)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Step 6: Compute Class Weights (inverse of class frequency)
labels_all = dataset.data['Label'].apply(lambda x: np.array([int(i) for i in x[1:-1].split(',')]))
label_sums = np.sum(np.stack(labels_all), axis=0)
class_freq = label_sums / len(dataset)
class_weights = torch.tensor(1.0 / (class_freq + 1e-5), dtype=torch.float32).to(device)

# Step 7: Define Optimizer and Loss Function
model = ModifiedMobileNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = FocalLoss(gamma=2., alpha=0.25, class_weights=class_weights)

# Step 8: Train the Model with Early Stopping
num_epochs = 30
patience = 15
best_val_loss = float('inf')
epochs_since_improvement = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement >= patience:
        print(f"Stopping early at epoch {epoch+1}")
        break

# Step 9: Evaluate the Model
model.eval()
predictions = []
targets = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions.append(outputs.cpu().numpy())
        targets.append(labels.cpu().numpy())

predictions = np.concatenate(predictions)
targets = np.concatenate(targets)

threshold = 0.4
predictions = (predictions > threshold).astype(int)

deformities_list = ['flatback', 'forwardhead', 'genuvalgum', 'genuvarum', 'kyphosis', 'lordosis', 'normal',
                    'scoliosis', 'toeout', 'torticolis', 'unevenhip', 'unevenshoulders']

print(classification_report(targets, predictions, target_names=deformities_list))

# Step 10: Plot Loss
plt.figure(figsize=(12,6))
plt.plot(range(1, len(train_losses)+1), train_losses, label="Training Loss")
plt.plot(range(1, len(val_losses)+1), val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.legend()
plt.show()

Streaming output truncated to the last 5000 lines.
/content/images/IMG_0214.JPG
[0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
/content/images/IMG_1357.JPG
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_1935.JPG
[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_0722.JPG
[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_2271.JPG
[0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_0839.JPG
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
/content/images/IMG_1679.JPG
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
/content/images/IMG_0963.JPG
[0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.]
/content/images/IMG_0301.JPG
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_0219.JPG
[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_1657.JPG
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1.]
/content/images/IMG_0558.JPG
[0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.]
/content/images/IMG_0694.JPG
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
/content/images/IMG_0450.JPG
[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
/content/im